# Complicated array type parameters:  {T, N, Array{T,N}} ?
Why do we keep seeing this pattern in type parameters? 

Inspired by [Tim Holy's keynote at JuliaCon 2016](https://www.youtube.com/watch?v=fl0g9tHeghA)

## 1. Reminder: type parameters

In [ ]:
struct Foo{S,T,U}
    x::S
    y::T
    z::U
end

In [ ]:
a = Foo(2, 2.0, "two")

In [ ]:
typeof(a)

In [ ]:
dump(a)

## 2. Putting information about the type of the contents of Foo allows us to dispatch on the contents:

In [ ]:
import Base.*

# This doesn't work because we need the "where" syntax
*(a::Foo{S,T,U},b::Foo{S,T,U}) = Foo(a.x+b.x, a.y+b.y, a.z+b.z)

In [ ]:
*(a::Foo{S,T,U}, b::Foo{S,T,U}) where {S,T,U} = Foo(a.x * b.x, a.y * b.y, a.z * b.z)

In [ ]:
# This includes 
# *(a::Foo{Int64,Float64,String}, b::Foo{Int64,Float64,String}) = Foo(a.x*b.x, a.y*b.y, a.z*b.z)

# Recall that * on strings does string concatenation:

In [ ]:
"ab" * "cd"

In [ ]:
a * a

In [ ]:
dump(a * a)

In [ ]:
b = Foo( ones(Int,2,2) , eye(Int,2), 100)

In [ ]:
b * b

## 3. Putting information about how an object is used is equally valuable 

In [ ]:
# Build our own pretend matrix of twos, storing only the size

struct Twos{T,N} <: AbstractArray{T,N}
   size :: NTuple{N,Int}
end

In [ ]:
one(Int)

In [ ]:
one(Float64)

In [ ]:
one(Rational{Int})

In [ ]:
one(BigFloat)

In [ ]:
one(Complex{Int64})

Defining the following two methods, `getindex` and `size`, is sufficient to endow the type `Twos` with array-like behavior ("array semantics"):

In [ ]:
Base.getindex(A::Twos{T,N}, i::Int...) where {T,N} = 2*one(T)   

Base.size(A::Twos) = A.size

In [ ]:
Twos(::Type{T}, i::Vararg{Int,N}) where {T,N} = Twos{T,N}(i)

Twos(i::Vararg{Int,N}) where {N} = Twos{Int,N}(i)

In [ ]:
# primitive constructor
Twos{Int,2}((3, 4))

In [ ]:
# convenience constructor
Twos(Float64, 3, 4)

In [ ]:
(√).(Twos(BigInt,3,4))

In [ ]:
# more convenient
Twos(3,4)

In [ ]:
dump(ans)

In [ ]:
a=Twos(3,5)
Foo(a,1,1)

In [ ]:
dump(ans)

## 4. Reshape as an example

In [ ]:
1:12

In [ ]:
dump(ans)

In [ ]:
collect(1:12)

In [ ]:
v = [1:12;] ## equivalent to collect(1:12)

In [ ]:
dump(v)

What happens when we reshape this array?

In [ ]:
A = reshape(v, 2, 6)

In [ ]:
B = copy(A)

In [ ]:
A2 = reshape(v, 3, 4)

These objects share data:

In [ ]:
v[5] = 100

In [ ]:
v

In [ ]:
A

In [ ]:
A2

In [ ]:
A2[3,4] = -10

In [ ]:
v

In [ ]:
A

## Reshaping a range

In [ ]:
r = 1:12

In [ ]:
dump(r)

In [ ]:
r.start, r.stop

In [ ]:
r + r

In [ ]:
dump(ans)

`r` behaves like a 1D array. Manipulating it often returns a standard array:

In [ ]:
r[3]

In [ ]:
dump(r)

In [ ]:
v2 = r.^2

Here, a *new* object was created.

In [ ]:
v2[3] = 400

In [ ]:
v2

In [ ]:
r[3]

Note that `r` is immutable (cannot be modified):

In [ ]:
r[3] = 5

In [ ]:
r

## Reshaping a range

What happens if we reshape a range? We would like the result to behave like an array. However, the underlying "data" doesn't exist in memory. Nonetheless, Julia can handle this:

In [ ]:
r = 1:12

In [ ]:
A = reshape(r, 3, 4)

In [ ]:
B = reshape(r,2,2,3)

In [ ]:
A = reshape([1:12;],3,4)

In [ ]:
dump(A)

This might have created a new array object, materialized in memory:

In [ ]:
A2 = reshape(collect(r), 3, 4)

But in fact, Julia does something sneakier: it creates an object that *behaves like an array*, but reuses the range object as the underlying substrate:

In [ ]:
dump(A)

In [ ]:
A.parent, A.dims

In [ ]:
dump(typeof(A))

In [ ]:
A = reshape(r, 3, 4)

The fact that the type of `A` is a subtype of `AbstractArray{Int64, 2}` is what shows us that Julia will treat the object *as if it were a matrix of `Int64`*. Some operations work correctly, while others don't:

In [ ]:
svdvals(A)

In [ ]:
@which svdvals(A)

In [ ]:
eigvals(A)

In [ ]:
dump(A)

The remaining type parameters of a `ReshapedArray` correspond to the internal representation; in this case, `UnitRange{Int64}` refers to the fact that the underlying data is coming from the `Range` object `r`. The tuple `dims` stores the information about the size of the reshaped array that Julia uses to treat `A` as an array:

In [ ]:
size(A)

In [ ]:
A + A  # this could have been optimized better, but at least it works

In [ ]:
reshape(2:2:24,3,4)

## MappedArrays

Suppose we need to sometimes calculate the pointwise square root of an array, but you don't know in advance where the evaluation will be. It is excessive to store an entire copy of the array with the square root already taken; also, you may wish to modify the underlying array.

In [ ]:
# Pkg.add("MappedArrays")
using MappedArrays

In [ ]:
M = reshape([1:12;], 3, 4)

Let's map the array with the square root function:

In [ ]:
M2 = mappedarray(√, M)

In [ ]:
dump(M2)

In [ ]:
typeof(M2) <: AbstractArray{Float64, 2}

Again, `M2` looks to Julia like a matrix of `Float64`, as given by the first two type parameters, but in fact, the underlying data is still the original matrix of `Int`s. When and only when an indexing operation occurs, the `sqrt` function is applied. (Note that when we display the array, an indexing operation is indeed occurring.)

We can even modify the underlying array:

In [ ]:
M[1, 2] = 100
M2[1, 2] 

Here, `M2[1, 2]` is automatically "updated", since it actually accesses M[1, 2] internally at every indexing operation

However, we cannot update `M2` directly, since it is a `ReadonlyMappedArray`:

In [ ]:
M2[1, 3] = 100

To do so requires telling Julia what the inverse function is to map back from the `MappedArray` to the original data:

In [ ]:
square(x) = x^2

M3 = mappedarray((√, square), M)

In [ ]:
M3[1,3] = 11

In [ ]:
M

## Mapping a reshaped array

What happens if we map a reshaped array?

In [ ]:
r = 1:12
A = reshape(r, 3, 4)
M = mappedarray(√, A)

In [ ]:
dump(M)

Here we see from the third type parameter that the underlying data is a `ReshapedArray`.

In [ ]:
M2 = mappedarray((√, square), A)

In [ ]:
M2[1, 2] = 11

# Views

A "view" into a matrix is another good example of a Holy type:

In [ ]:
A = reshape([1:12;], 3, 4)

In [ ]:
B = view(A, 1:2, 2:4)

This behaves like an `Int`eger matrix, with underlying data from an integer matrix, and indexed by `UnitRange` objects. The final type parameter indicates whether the resulting object can indexed linearly (i.e. if it is consecutive in memory).

# Summary

Tim Holy: 
> Arrays are natural for lazy evaluation. 

The type system is used to keep track of the contortions you do on the data without actually doing it on the data -- this is lazy evaluation.